#Build The Neural Network

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

##Get Device for Training

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


##Define the Class
- We define our neural network by subclassing `nn.Module`
- and initialize the neural network layers in `__init__`
- Every `nn.Module` subclass implements the operations on input data in the `forward` method.

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(logits)
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[ 0.0142,  0.0546, -0.0317,  0.0688, -0.0032, -0.0743,  0.0005, -0.0382,
          0.0077,  0.0009]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.1014, 0.1055, 0.0968, 0.1070, 0.0996, 0.0928, 0.1000, 0.0962, 0.1007,
         0.1000]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([3], device='cuda:0')


##Model Layers

In [10]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


###nn.Flatten
- convert each 2D 28x28 image into a 1D array of 784 pixel values (the minibatch dimension is maintained).

In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


###nn. Linear
- applies a linear transformation on the input using its stored weights and biases.

In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


###nn.ReLU
- introduces nonlinerity at the model helping neural networks learn a wide variety of phenomena.

In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLu: {hidden1}")

Before ReLU: tensor([[-0.0562,  0.4282,  0.3564,  0.3420, -0.6055,  0.0562, -0.0653,  0.0328,
         -0.2504, -0.1781,  0.0097,  0.4056, -0.2581,  0.1167, -0.2946,  0.2363,
          0.3781,  0.1934, -0.3982, -0.3814],
        [ 0.0128,  0.7109,  0.1790,  0.1363, -0.5430,  0.0968,  0.1184, -0.1819,
         -0.0378,  0.2445,  0.2409,  0.5428, -0.2593,  0.3276, -0.5360,  0.1658,
          0.3361,  0.1537, -0.4363, -0.0772],
        [ 0.1081,  0.3468,  0.3230,  0.2280, -0.3879,  0.0238, -0.2447,  0.1580,
         -0.4345, -0.1467,  0.1188,  0.5381, -0.3494,  0.3559, -0.0332,  0.0804,
          0.3833,  0.1036, -0.0262, -0.0773]], grad_fn=<AddmmBackward0>)


After ReLu: tensor([[0.0000, 0.4282, 0.3564, 0.3420, 0.0000, 0.0562, 0.0000, 0.0328, 0.0000,
         0.0000, 0.0097, 0.4056, 0.0000, 0.1167, 0.0000, 0.2363, 0.3781, 0.1934,
         0.0000, 0.0000],
        [0.0128, 0.7109, 0.1790, 0.1363, 0.0000, 0.0968, 0.1184, 0.0000, 0.0000,
         0.2445, 0.2409, 0.5428, 0.0000, 0.3276, 0.00

###nn.Sequential 
- an ordered container of modules.


In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

###nn.Softmax
- scales to values [0, 1] representing the model's predicted probabilities for each class.
- `dim`: the dimension along which the values must some to 1

In [19]:
softmax = nn.Softmax(dim=1)
pred_proab = softmax(logits)
print(pred_proab)

tensor([[0.0915, 0.1025, 0.0833, 0.1023, 0.0675, 0.0855, 0.1082, 0.1309, 0.1218,
         0.1065],
        [0.0999, 0.0998, 0.0842, 0.0952, 0.0859, 0.0878, 0.1154, 0.1179, 0.1157,
         0.0983],
        [0.0915, 0.1006, 0.0813, 0.0931, 0.0763, 0.0765, 0.1136, 0.1284, 0.1277,
         0.1109]], grad_fn=<SoftmaxBackward0>)


##Model Parameters
- Subclass of `nn.Module` can automatically track all fields defined inside your model object, and make all parameters accessible using your model's `parameters()` or `named_parameters()` methods.

In [20]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0239, -0.0185,  0.0179,  ..., -0.0282, -0.0224,  0.0094],
        [ 0.0239,  0.0270,  0.0182,  ...,  0.0078,  0.0103, -0.0027]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0346,  0.0098], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0031, -0.0244,  0.0162,  ...,  0.0060,  0.0180, -0.0043],
        [-0.0029, -0.0195,  0.0004,  ..., -0.0396, -0.0377, -0.0190]],
       device='cuda:0', grad_fn=<SliceBa